In [ ]:
import pymysql
import csv
import pandas as pd
import numpy as np
db = pymysql.connect(host='localhost', user='root', passwd='***')
cursor = db.cursor()
cursor.execute('CREATE DATABASE loan_stat;')
cursor.execute("Show databases;")
cursor.execute('USE loan_stat;')

In [ ]:
#check if there is any difference between the headers of datafiles for different years
names = ['LoanStats_2007_2011', 'LoanStats_2012_2013', 'LoanStats_2014', 'LoanStats_2015']
i=0
headers=[]
files = []
for name in names:
    files.append(name+'.csv')
    with open(files[i], newline='') as f:
        reader = csv.reader(f) 
        headers.append(next(reader))
        

for i in range(1, len(headers)):
    print(set(headers[i])-set(headers[i-1]))
    #print(headers[i]==headers[i-1])
#headers[0]

## create a table named loans_tot

In [ ]:
#sql query for creating new table

data = pd.read_csv('LoanStats_2007_2011.csv', encoding="ISO-8859-1")
dtypes = data.dtypes

dtype_li = list()
attr_li = list()

for attr, py_dtype in dtypes.iteritems() :
    if py_dtype == 'int64':
        sql_dtype = 'int(11)'
    if py_dtype == 'float64':
        sql_dtype = 'float(15,4)'
    if py_dtype == 'O':
        sql_dtype = 'varchar(100)'
    if attr == 'desc':
        attr = "description"
        sql_dtype = 'varchar(5000)'
    if attr == 'verification_status_joint':
        sql_dtype = 'varchar(100)'
    attr_li.append(attr)
    dtype_li.append(sql_dtype)

In [ ]:
# create table

query = 'CREATE TABLE loans ('
for i in range(len(attr_li)):
    query = query+'{} {}, '.format(attr_li[i], dtype_li[i])
query = query+'PRIMARY KEY (id));'
#print(query)

cursor.execute(query)

## Insert Data From Multiple csv files

In [ ]:
# sql query for importing data
# dealing with empty fields in csv data by convering them to NULL in mysql (refer to 'MySQL load NULL values from CSV data'--stackoverflow)

set_null_sql = '(id,'
for attr in attr_li[1:]:
    set_null_sql += ' @v'+attr+','
set_null_sql = set_null_sql[:-1]+') SET'
for attr in attr_li[1:]:
    set_null_sql += ' '+attr+' = nullif(@v'+attr+",''),"
set_null_sql = set_null_sql[:-1]+';'

#print(set_null_sql)

In [ ]:
name_li = ['LoanStats_2007_2011', 'LoanStats_2012_2013', 'LoanStats_2014', 'LoanStats_2015']
infile_path =  'C:/.../p2pLending_project/{}.csv'
outfile_path = 'C:/ProgramData/MySQL/MySQL Server 5.7/Uploads/{}.csv'

sql = r"""LOAD DATA INFILE '{}' INTO TABLE loans FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\r\n' IGNORE 1 LINES""" + set_null_sql

for name in name_li:
    # convert csv files encoding to utf-8 and move files to the directory specified by secure-file-priv
    # use SHOW VARIABLES LIKE "secure_file_priv"; (in mysql) to see the directory from which you can load files using LOAD DATA INFILE
    with open(infile_path.format(name),'r', encoding='utf-8', errors='ignore', newline='') as infile, open(outfile_path.format(name),'w', newline='') as outfile:
        inputs = csv.reader(infile)
        outputs = csv.writer(outfile)
        for row in inputs:
            outputs.writerow(row)
    
    cursor.execute(sql.format(outfile_path.format(name)))
    print(name)

In [ ]:
cursor.close()
db.close()